In [1]:
import oci #versão do OCI deve ser: {oci.__version__} 2.66.1+preview.1.4774
import os
import base64
import ocifs
import PIL.Image as Image
import io
import re
import numpy as np 
import pandas as pd

config = oci.config.from_file('~/.oci/config')
ai_vision_client = oci.ai_vision.AIServiceVisionClient(config=config)

endpoint = "https://language.aiservice.us-ashburn-1.oci.oraclecloud.com"
ai_client = oci.ai_language.AIServiceLanguageClient(config, service_endpoint=endpoint)

fs = ocifs.OCIFileSystem()
object_storage_client = oci.object_storage.ObjectStorageClient(config)

img_list = fs.glob("oci://<nome-do-bucket>@<namespace>/*.jpg")

def valida_documento(ner_inference_result):
    lista_check_1 = ['Pagamento', 'Valor', 'Data']
    lista_check_2 = ['Pix', 'Valor', 'Data']
    lista_individual = []
    
    for j in range (0, len(ner_inference_result.data.documents[0].entities)):
        lista_individual.append(ner_inference_result.data.documents[0].entities[j].type)
                    
    validade_pagamento =  all(x in lista_individual for x in lista_check_1)
    validade_pix =  all(x in lista_individual for x in lista_check_2)
    
    diff = np.setdiff1d(lista_check_1, lista_individual)
    
    if validade_pagamento:
        validade = True
        diff = np.setdiff1d(lista_check_1, lista_individual)  
    
    elif validade_pix: 
        validade = True
        diff = np.setdiff1d(lista_check_2, lista_individual)
    
    else: 
        validade = False             
    
    return validade, diff

def formata_valor(valor):
     try:
        valor_tratado = re.findall('[0-9]+,[0-9]+.[0-9]+|[0-9]+.[0-9]+,[0-9]+|[0-9]+.[0-9]+|[0-9]+,[0-9]+', valor)
        valor_tratado = valor_tratado[0]
        
        return valor_tratado
    
     except:
        valor_tratado = '0'
        
        return valor_tratado

name_list = []
for i, item in enumerate(img_list):
    name = item.rsplit('/', 1)[1]
    name_list.append(name)

In [2]:
dataset = pd.DataFrame(columns=['Código', 'Valor (R$)', 'Validade', 'Informação Faltante'])

In [ ]:
for i, file in enumerate(img_list):
    try:
        codigo = name_list[i]

        valor = []
        data = []
        pagamento = []

        with fs.open(file) as f:
            content = f.read()

            encoded_string = base64.b64encode(content, altchars=None)
            decoded_string = encoded_string.decode("utf-8", "ignore")

            analyze_document_response = ai_vision_client.analyze_document(
                    analyze_document_details=oci.ai_vision.models.AnalyzeDocumentDetails(
                    compartment_id = "ocid1.compartment.oc1..aaaaaaaaxxxxxxxxxxxxxxxxxx5xxxxxxxxxxxxxxxxx6xxxxxxxx", #Colocar seu compartment
                    features=[oci.ai_vision.models.DocumentTextDetectionFeature(feature_type="TEXT_DETECTION")],
                    language='ENG',
                    document=oci.ai_vision.models.InlineDocumentDetails(source="INLINE",data=decoded_string))
                    )

            words = str([word.text for page in analyze_document_response.data.pages for word in page.words])
            name = f"{name_list[i]}.txt"  

            #words_str = words.decode()

            words_str_1 = words.replace("',",'')
            words_str_2 = words_str_1.replace("'",'')
            words_str_3 = words_str_2.replace("[",'')
            words_str_4 = words_str_3.replace("]",'')

        ner_text_for_testing = oci.ai_language.models.BatchDetectLanguageEntitiesDetails(
                endpoint_id = "ocid1.ailanguageendpoint.oc1.iad.aaaaaaaaxxxxxxxxxxxxxxxxxx5xxxxxxxxxxxxxxxxx6xxxxxxxx", #Colocar seu endpoint
                documents = [oci.ai_language.models.TextDocument(key = "1",text = words_str_4)]
                )

        ner_inference_result = ai_client.batch_detect_language_entities(ner_text_for_testing)
        
        validade, info_faltante = valida_documento(ner_inference_result)

        for j in range (0, len(ner_inference_result.data.documents[0].entities)):

            entidade = ner_inference_result.data.documents[0].entities[j].type
            texto = ner_inference_result.data.documents[0].entities[j].text

            if entidade == 'Valor':
                valor_formatado = formata_valor(texto)
                valor.append(valor_formatado)      

            elif entidade == 'Data':
                data.append(texto)

            elif entidade == 'Pagamento':
                pagamento.append(texto)

            elif entidade == 'Pix':
                pagamento.append(texto)

        dataset = dataset.append({'Código': codigo, 'Valor (R$)' : valor[0],  
                                'Validade' : validade, 'Informação Faltante' : info_faltante}, ignore_index=True)

    except:
        print(f'Amostra inválida, código {codigo}')

In [ ]:
dataset